In [13]:
import os
import random
import numpy as np
from math import sqrt
import collections
random.seed(1117)


In [14]:
# If set K to 2, then the original 6 labels will be divided into two labels: static state and active state

K = 6
print('K is {}'.format(K))
TRAIN_DIR = 'UCI HAR Dataset/train'
TEST_DIR = 'UCI HAR Dataset/test'


def read_a_file(file):
    """
    read_a_file: Read the data file, Each row is corresponding to one subject's motion. . Return a numpy array
    file: the file path
    
    """
    f = open(file, 'r', encoding='utf-8')
    data = []
    for line in f:
        line = line.strip().split()
        x = [float(v) for v in line]
        data.append(x)
    f.close()
    data = np.array(data)
    return data


def read_dataset(data_dir):
    """
    read_dataset: Read X and y form 'UCI HAR Dataset/train' or 'UCI HAR Dataset/test' . The returned X and y are used
    as training or testing data set.

    data_dir: file address 'UCI HAR Dataset/train' or 'UCI HAR Dataset/test'
     X (A n * f matrix,where n is the number of samples, f is the number of features),
     y (A n*1 matrix where n is corresponding to the number of labels)
    """
    print('Read data from {}'.format(data_dir))
    kind = 'train' if 'train' in data_dir else 'test'

    # X has 2 parts: for train, it comes form X_train.txt and Inertial Signals
    
    # Part 1. X_train.txt or X_test.txt
    X1 = read_a_file(os.path.join(data_dir, 'X_{}.txt'.format(kind)))
    print('The data shape of X_{}.txt is {} × {}'.format(kind, X1.shape[0], X1.shape[1]))

   
    # Part 2. Inertial Signals
    # IS_dir = os.path.join(data_dir, 'Inertial Signals')
    # X2 = []
    # for file_name in os.listdir(IS_dir):
    #     file_path = os.path.join(IS_dir, file_name)
    #     tmp = read_a_file(file_path)
    #     X2.append(tmp)
    # X2 = np.concatenate(X2, axis=1)
    # print('The data shape of Inertial Signals is {} × {}'.format(X2.shape[0], X2.shape[1]))

    # X = np.concatenate([X1, X2], axis=1)
    
    X = X1

    # Read the y data
    y = read_a_file(os.path.join(data_dir, 'y_{}.txt'.format(kind)))
    y = y.reshape(-1)
    y = [int(v) for v in y]
    return X, y


def euclidean_distance(a, b):
    """
    euclidean_distance: Computing the euclidean distance between two given points
    a: One point
    b: Another point
    
    """
    
    d = len(a)
    cur_sum = 0
    for i in range(d):
        cur_sum += (a[i] - b[i]) ** 2
    return sqrt(cur_sum)


def center_point(points):
    """
    center_point: Computing the center point of data points. Each dimension of the center point is
    the mean of the points of the same dimension.
    points: a list of points
    
    """
    n = len(points)
    d = len(points[0])

    center = []

    for i in range(d):
        cur_sum = 0  # dimension sum
        for p in points:
            cur_sum += p[i]

        # average of each dimension
        center.append(cur_sum / float(n))

    return center


def update_centers(data, assignments):
    """
    update_centers:According to each data point and their assignment, compute a center point for each kind of assignment
    It returns to a list containing k centers
    data: All data points
    assignments: The assignments for all points, e.g., data[i] has the assignment of assignments[i]
   
    """
    ass_2_points = {}
    for assignment, point in zip(assignments, data):
        if assignment not in ass_2_points:
            ass_2_points[assignment] = [point]
        else:
            ass_2_points[assignment].append(point)

    centers = []
    for points in ass_2_points.values():
        centers.append(center_point(points))

    return centers


def assign_points(data, centers):
    """
    assign_points:The assignments of all points. centers contain several center points. 
    For each point in the data, find the closest center point and its index,
    use this index as the assignment of this point.
    data: All data points
    centers: Center points
    
    """
    assignments = []
    for point in data:
        closest = float('inf')  # positive infinity
        closest_index = 0
        for i in range(len(centers)):
            val = euclidean_distance(point, centers[i])
            if val < closest:
                closest = val
                closest_index = i
        assignments.append(closest_index)
    return assignments


def kmeans_plusplus(data, k):
    """
    kmeans_plusplus: Choose k points in data as k center points using K-means++ algorithm.
    It returns to a list of k center points
    data: All data points
    k: The number of center points

    """
    centers = list()
    centers.append(random.choice(data))     # Choose the first center point
    # find left k-1 center points
    for i in range(1, k):
        d = [min([euclidean_distance(x, c) for c in centers]) for x in data]
        # use Roulette method to choose next center point
        d = [p/sum(d) for p in d]
        s = 0
        for j in range(len(d)):
            d[j] += s
            s += d[j]
        r = random.random()
        for j, p in enumerate(d):
            if r < p:
                centers.append(data[j])
                break
    return centers


def compare_two_assignments(a1, a2):
    """
    compare_two_assignments: numbers of different values. 
    :param a1: One assignment
    :param a2: Another assignment
    
    """
    diff = 0
    for i, j in zip(a1, a2):
        if i != j:
            diff += 1
    return diff



K is 6


In [17]:
def run_k_means(X, k):
    """
    run_k_means: Run the K-means algorithm. Partition the X into k clusters. It returns to the assignments of X.
    X: All data points.
    k: The final cluster number.
    
    """
    print('Start runing K-means...')
    centers = kmeans_plusplus(X, k)
    times = 10000
    it = 1
    assignments = assign_points(X, centers)
    while it <= times:
        print('Begin {} iteration...'.format(it))
        centers = update_centers(X, assignments)
        old_assignments = assignments[:]
        assignments = assign_points(X, centers)
        it += 1
        cur_diff = compare_two_assignments(old_assignments, assignments)
        if cur_diff == 0:
            break
    print('K-means done!')
    return assignments, centers

if __name__ == '__main__':
    train_X, train_y = read_dataset(TRAIN_DIR)
    print()
    test_X, test_y = read_dataset(TEST_DIR)
    print()

    if K == 2:
        train_y = [0 if v in [1, 2, 3] else 1 for v in train_y]
        test_y = [0 if v in [1, 2, 3] else 1 for v in test_y]

    assignments, centers = run_k_means(train_X, K)

    print(len(centers))
   # print(assignments[:1000])

    # The points of the same assignment is a class of points. Use the majority of true labels of each class as the
    # label of this class. In the test phase, for each test point, computing the distance between it and the centers
    # points, and choose the index of the closest center point as the class of this test point. Then using the label of
    # this class as the predicted label of this test point.

    # ass_2_label_lists is the list of true labels for points with the same assignment
    ass_2_label_lists = {}
    for i, ass in enumerate(assignments):
        label = train_y[i]
        if ass not in ass_2_label_lists:
            ass_2_label_lists[ass] = [label]
        else:
            ass_2_label_lists[ass].append(label)

    # ass_2_label is the label for each assignment, by counting the most frequent labels in the label lists
    # of the assignment
    ass_2_label = {}
    for ass in ass_2_label_lists:
        ct = collections.Counter(ass_2_label_lists[ass])
        label = ct.most_common(1)[0][0]
        ass_2_label[ass] = label

        print('ass = {}, size = {}, label = {}'.format(ass, len(ass_2_label_lists[ass]), label))

    print(ass_2_label)

    # Test on train data
    accuracy = 0
    for i, x in enumerate(train_X):
        # compute the assignment for this x
        predicted_label = ass_2_label[assignments[i]]
        if predicted_label == train_y[i]:
            accuracy += 1
    accuracy /= len(train_y)
    print('When  K={}, The error rate for trainning dataset is {}'.format(K, 1-accuracy))
    
     # Test on test data
    accuracy = 0
    for i, x in enumerate(test_X):
        # compute the assignment for this x
        closest = float('inf')
        assignment = 0
        for i in range(len(centers)):
            val = euclidean_distance(x, centers[i])
            if val < closest:
                closest = val
                assignment = i
        predicted_label = ass_2_label[assignment]
        if predicted_label == test_y[i]:
            accuracy += 1

    accuracy /= len(test_y)
    print('When K={}, the error rate for testing dataset is {}'.format(K, 1-accuracy))

   

Read data from UCI HAR Dataset/train
The data shape of X_train.txt is 7352 × 561
The data shape of X is 7352 × 561
The data shape of y is 7352 × 1

Read data from UCI HAR Dataset/test
The data shape of X_test.txt is 2947 × 561
The data shape of X is 2947 × 561
The data shape of y is 2947 × 1

Start runing K-means...
Begin 1 iteration...
Begin 2 iteration...
Begin 3 iteration...
Begin 4 iteration...
Begin 5 iteration...
Begin 6 iteration...
Begin 7 iteration...
Begin 8 iteration...
Begin 9 iteration...
Begin 10 iteration...
Begin 11 iteration...
Begin 12 iteration...
Begin 13 iteration...
Begin 14 iteration...
Begin 15 iteration...
Begin 16 iteration...
Begin 17 iteration...
Begin 18 iteration...
Begin 19 iteration...
Begin 20 iteration...
Begin 21 iteration...
Begin 22 iteration...
Begin 23 iteration...
K-means done!
4
ass = 0, size = 1939, label = 5
ass = 1, size = 852, label = 5
ass = 2, size = 1288, label = 6
ass = 3, size = 3273, label = 1
{0: 5, 1: 5, 2: 6, 3: 1}
When  K=6, The er